In [4]:
include("../MyReverseDiff.jl")
include("../MyEmbedding.jl")
include("../MyMlp.jl")

using .MyReverseDiff
using .MyEmbedding
using .MyMlp
using JLD2
using Printf
using BenchmarkTools
using LinearAlgebra
using Distributions
using Random
using MLDatasets
using Plots
using Statistics
using DataFrames
using MLDataUtils

## Przygotowanie danych IMDB

In [5]:
X_train = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "X_train"));
y_train = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "y_train"));
X_test = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "X_test"));
y_test = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "y_test"));
input_size = size(X_train, 1) # Liczba cech

17703

## Przygotowanie modelu

In [6]:
#   Definicja rozmiarów modelu
input_size = size(X_train, 1) # Liczba cech
hidden_size = 8
output_size = 1
batch_size = 64

#   Inicjalizacja modelu (Chain) (raz)
model = Chain(
    Dense(input_size, hidden_size, relu; weight_init=xavier_uniform,  name="layer1"),
    Dense(hidden_size, output_size, σ; weight_init=xavier_uniform, name="layer2")
)

#   Utworzenie początkowych węzłów Constant dla danych wejściowych i etykiet
x_input_node = Constant(zeros(Float32, input_size, batch_size))
y_label_node = Constant(zeros(Float32, output_size, batch_size))

#   Budowanie grafu treningowego
loss_node, model_output_node, order = build_graph!(model, binarycrossentropy, x_input_node, y_label_node; loss_name="loss")

optimizer_state = setup_optimizer(Adam(), model)

AdamState(Adam(0.001f0, 0.9f0, 0.999f0, 1.0f-8), Dict{String, Matrix{Float32}}("layer2_b" => [0.0;;], "layer2_w" => [0.0 0.0 … 0.0 0.0], "layer1_b" => [0.0; 0.0; … ; 0.0; 0.0;;], "layer1_w" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]), Dict{String, Matrix{Float32}}("layer2_b" => [0.0;;], "layer2_w" => [0.0 0.0 … 0.0 0.0], "layer1_b" => [0.0; 0.0; … ; 0.0; 0.0;;], "layer1_w" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]), 0, Tuple{String, Variable}[("layer1_w", var layer1_w
 ┣━ ^ 8×17703 Matrix{Float32}
 ┗━ ∇ 8×17703 Matrix{Float32}), ("layer1_b", var layer1_b
 ┣━ ^ 8×1 Matrix{Float32}
 ┗━ ∇ 8×1 Matrix{Float32}), ("layer2_w", var layer2_w
 ┣━ ^ 1×8 Matrix{Float32}
 ┗━ ∇ 1×8 Matrix{Float32}), ("layer2_b", var layer2_b
 ┣━ ^ 1×1 Matrix{Float32}
 ┗━ ∇ 1×1 Matrix{Float32})])

##  Trening modelu

In [4]:

# epochs = 5


# for epoch in 1:epochs
#     # --- Tasowanie zbioru treningowego NA NOWO w każdej epoce ---
#     permutation = randperm(size(X_train, 2))
#     X_train_shuffled_epoch = X_train[:, permutation]
#     y_train_shuffled_epoch = y_train[:, permutation]
#     num_batches = ceil(Int, size(X_train, 2) / batch_size)

#     loss_value = 0.0

#     t = @elapsed begin

#     for i in 1:num_batches

#         start_idx = (i - 1) * batch_size + 1
#         end_idx = min(i * batch_size, size(X_train, 2))
#         x_batch = X_train_shuffled_epoch[:, start_idx:end_idx]
#         y_batch = y_train_shuffled_epoch[:, start_idx:end_idx]

#         current_batch_size = size(x_batch, 2)
#         view(x_input_node.output, :, 1:current_batch_size) .= x_batch
#         view(y_label_node.output, :, 1:current_batch_size) .= y_batch


#         forward!(order)

#         backward!(order)

#         step!(optimizer_state)
#         loss_value += loss_node.output

#     end
# end
#     avg_loss_epoch = loss_value / num_batches

#     println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f)", epoch, t, avg_loss_epoch))
# end

In [7]:
using Printf # Dla @sprintf

# ... (Twój istniejący kod: definicja modelu, x_input_node, y_label_node, build_graph!, setup_optimizer)

epochs = 5

println("--- Rozpoczynam profilowanie treningu ---")

for epoch in 1:epochs
    # --- Tasowanie zbioru treningowego NA NOWO w każdej epoce ---
    permutation = randperm(size(X_train, 2))
    X_train_shuffled_epoch = X_train[:, permutation]
    y_train_shuffled_epoch = y_train[:, permutation]
    num_batches = ceil(Int, size(X_train, 2) / batch_size)

    loss_value = 0.0

    # Profilowanie pętli batchowej
    # Użyj @time do pomiaru czasu i alokacji wewnątrz pętli.
    # Ważne: pierwsze uruchomienie `@time` może być wolniejsze z powodu kompilacji JIT.
    # Zrób jedno "suche" uruchomienie przed właściwym profilowaniem, jeśli to konieczne.
    println("\nEpoch: $epoch")
    total_batch_time = 0.0
    total_batch_alloc = 0
    total_batch_gc_time = 0.0

    for i in 1:num_batches
        start_idx = (i - 1) * batch_size + 1
        end_idx = min(i * batch_size, size(X_train, 2))
        x_batch_view = view(X_train_shuffled_epoch, :, start_idx:end_idx)
        y_batch_view = view(y_train_shuffled_epoch, :, start_idx:end_idx)

        current_batch_size = size(x_batch_view, 2)
        view(x_input_node.output, :, 1:current_batch_size) .= x_batch_view
        view(y_label_node.output, :, 1:current_batch_size) .= y_batch_view

        # Profilowanie pojedynczego kroku batcha
        # Użyj `@time` dla całego bloku operacji
        # Wynik `@time` to tuple: (time, bytes, gctime, compile_time, reclaim_ratio)
        # Bierzemy tylko czas i alokacje
        
        # Aby uzyskać czyste pomiary alokacji, najlepiej zrobić to dla jednego batcha
        # i wywołać `GC.gc()` przed każdym pomiarem, aby upewnić się, że mierzymy świeże alokacje.
        # W pętli treningowej to jednak zaburzyłoby realny czas wykonania.
        # Na potrzeby wstępnego profilowania, `@time` w pętli jest ok.

        stats = @timed begin # `timed` zwraca strukturę z wynikami, `time` tylko czas
            forward!(order)
            backward!(order)
            step!(optimizer_state) # Zakładam, że masz już zaimplementowane step!
        end
        loss_value += loss_node.output # Upewnij się, że loss_node.output jest odświeżane po forward

        total_batch_time += stats.time
        total_batch_alloc += stats.bytes
        total_batch_gc_time += stats.gctime
    end
    
    avg_loss_epoch = loss_value / num_batches

    println(@sprintf("Epoch: %d \tTrain: (l: %.4f) \tTotal Batch Time: %.4fs \tTotal Alloc: %s \tGC Time: %.4fs", 
                     epoch, avg_loss_epoch, total_batch_time, Base.format_bytes(total_batch_alloc), total_batch_gc_time))
end

println("\n--- Koniec profilowania treningu ---")

--- Rozpoczynam profilowanie treningu ---

Epoch: 1
Epoch: 1 	Train: (l: 0.6720) 	Total Batch Time: 4.3515s 	Total Alloc: 1.817 GiB 	GC Time: 0.3709s

Epoch: 2
Epoch: 2 	Train: (l: 0.5920) 	Total Batch Time: 0.6493s 	Total Alloc: 1.258 GiB 	GC Time: 0.3036s

Epoch: 3
Epoch: 3 	Train: (l: 0.4958) 	Total Batch Time: 0.6269s 	Total Alloc: 1.258 GiB 	GC Time: 0.2955s

Epoch: 4
Epoch: 4 	Train: (l: 0.4042) 	Total Batch Time: 0.6393s 	Total Alloc: 1.258 GiB 	GC Time: 0.2931s

Epoch: 5
Epoch: 5 	Train: (l: 0.3267) 	Total Batch Time: 0.6498s 	Total Alloc: 1.258 GiB 	GC Time: 0.2906s

--- Koniec profilowania treningu ---


##  Test modelu

In [8]:
# --- Test Evaluation ---

batch_size = 64
num_test_samples = size(X_test, 2)
num_batches = ceil(Int, num_test_samples / batch_size)
total_test_loss_sum = 0.0
total_correct_predictions = 0.0

t_test = @elapsed begin
    for i in 1:num_batches

        start_idx = (i - 1) * batch_size + 1
        end_idx = min(i * batch_size, num_test_samples)
        x_batch_test = X_test[:, start_idx:end_idx]
        y_batch_test = y_test[:, start_idx:end_idx]

        # Aktualna liczba próbek w bieżącym batchu (może być mniejsza dla ostatniego batcha)
        current_test_batch_size = size(x_batch_test, 2)

        view(x_input_node.output, :, 1:current_test_batch_size) .= x_batch_test
        view(y_label_node.output, :, 1:current_test_batch_size) .= y_batch_test

        forward!(order)

        predictions = view(model_output_node.output, :, 1:current_test_batch_size)


        batch_loss = loss_node.output
        
        total_test_loss_sum += batch_loss * current_test_batch_size # Sumuj stratę, uwzględniając rozmiar batcha

        # --- Oblicz dokładność na bieżącym batchu testowym ---
        # Dla klasyfikacji binarnej z progiem 0.5 (lub innym, w zależności od problemu)
        batch_accuracy = sum((predictions .> 0.5f0) .== y_batch_test) / current_test_batch_size
        total_correct_predictions += batch_accuracy * current_test_batch_size # Sumuj poprawne predykcje
    end
end

# --- Oblicz średnią stratę i średnią dokładność na całym zbiorze testowym ---
avg_test_loss = total_test_loss_sum / num_test_samples
avg_test_accuracy = total_correct_predictions / num_test_samples * 100.0

println(@sprintf("Test Loss (czas: %.2fs): %.4f", t_test, avg_test_loss))
println("Test Accuracy: $avg_test_accuracy %")


Test Loss (czas: 0.53s): 0.4062
Test Accuracy: 87.1 %
